In [10]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
file_loc = "../Matlab/EERdatabase.xlsx"

In [62]:
# basically after they read in the file they do some calculation and turn them
# into a file call data_vars.mat
# 

# replicates the reading phase

# read in the files
cpi = pd.read_excel(file_loc, sheet_name="cpi",engine="openpyxl") 

USD = pd.read_excel(file_loc, sheet_name="ner_eop",engine="openpyxl") 

gdp = pd.read_excel(file_loc, sheet_name="gdppc_ppp",engine="openpyxl") 

nfa = pd.read_excel(file_loc, sheet_name="nfa2gdp",engine="openpyxl") 

tot = pd.read_excel(file_loc, sheet_name="tot_all",engine="openpyxl") 

weights = pd.read_excel(file_loc, sheet_name="weights_static",engine="openpyxl") 

# pre cleaning / define some variables that would be useful
time_line = cpi.iloc[:,0]
countries = list(cpi.columns)[1:]

# the first columns is always time, we DO NOT need them in the calculation
cpi = cpi.iloc[:,1:].to_numpy()
gdp = gdp.iloc[:,1:].to_numpy()
USD = USD.iloc[:,1:].to_numpy()
nfa = nfa.iloc[:,1:].to_numpy()
tot = tot.iloc[:,1:].to_numpy()

# weights

# following their reindexing into alphabetic order
weights.columns = ["country", 
"time", 
"USA", 
"GBR", 
"DNK",
"NOR",
"SWE", 
"CHE",
"CAN",
"JPN",
"EA",
"AUS",
"NZL"]
weights.set_index("country", inplace=True)
weights = weights.reindex(index=sorted(weights.index), columns=sorted(weights.columns))
# they fill the na with 0 for weights
weights = weights.fillna(0)
weights.drop(columns="time", inplace=True)
weights = weights.to_numpy()


# calculations below 

# log of relative CPI
rpi = np.log(cpi)- np.log(cpi)@weights
#  log of USD rate (increase stands for appreciation of dom. currency)
ner = np.log(USD)@weights - np.log(USD)
# log of USD real rate (increase stands for appreciation)
rer = ner + rpi

# log of relative gdp per capita
rgdp = np.log(gdp) - np.log(gdp)@weights

# log of relative terms of trade

rtot = np.log(tot) - np.log(tot)@weights

# relative nfa (choose which one)
rnfa = nfa-nfa@weights


### in the original code they get rid of denmark 
rpi = np.delete(rpi ,3,1)  
ner = np.delete(ner,3,1)  
rer = np.delete(rer,3,1) 
rgdp = np.delete(rgdp,3,1) 
rtot = np.delete(rtot,3,1) 
rnfa = np.delete(rnfa, 3,1)
countries.remove("DNK")


array([[-1.03271253e-01, -3.07397836e-01,  6.01441784e-01, ...,
        -7.69705652e-02,  9.93912154e-03,  9.40729426e-02],
       [-1.13219324e-01, -3.12049083e-01,  6.25742343e-01, ...,
        -9.20247265e-02,  5.82297630e-03,  9.64138653e-02],
       [-1.27240753e-01, -3.16605058e-01,  6.44065588e-01, ...,
        -1.02491571e-01,  1.35362527e-03,  9.83187859e-02],
       ...,
       [-5.43782102e-01,  5.29893251e-01,  1.49333733e+00, ...,
        -4.10179039e-01, -3.28745624e-02, -6.74689751e-01],
       [-5.15658713e-01,  6.04556025e-01,  1.44121089e+00, ...,
        -4.06848535e-01, -6.51378048e-02, -7.26275910e-01],
       [-5.24849842e-01,  5.43937032e-01,  1.53743726e+00, ...,
        -4.50463664e-01, -8.94914139e-02, -7.18775474e-01]])